In [28]:
# Load SemEval dataset

from datasets import load_dataset

dataset = load_dataset("sem_eval_2010_task_8")
relation_names = dataset['train'].features['relation'].names                       # 19 relation classes
relation_name_to_idx = {name: idx + 1 for idx, name in enumerate(relation_names)}  # 1-indexed name-to-idx dict

In [31]:
def generate_COT_prompt(sentence, relation):
    
    task_summary = "You are an expert annotator for a natural language processing task, performing relation extraction on sentences. You have the following relation labels: Cause-Effect(e1,e2), Cause-Effect(e2,e1), Component-Whole(e1,e2), Component-Whole(e2,e1), Content-Container(e1,e2), Content-Container(e2,e1), Entity-Destination(e1,e2), Entity-Destination(e2,e1), Entity-Origin(e1,e2), Entity-Origin(e2,e1), Instrument-Agency(e1,e2), Instrument-Agency(e2,e1), Member-Collection(e1,e2), Member-Collection(e2,e1), Message-Topic(e1,e2), Message-Topic(e2,e1), Product-Producer(e1,e2), Product-Producer(e2,e1), Other."
    
    prompt = (f"{sentence}\n"
             f"In paragraph form, discuss in-depth the two entities in the above sentence and how they interact. "
             f"Then explain why {relation} is the correct relation extraction label for the sentence.")

    return [{"role": "system", "content": task_summary}, {"role": "user", "content": prompt}]

In [41]:
# Send explanation requests to a json file.

import os
import json
from groq import Groq

client = Groq(api_key = os.getenv('GROQ_API_KEY'))

cot_filename = "cot_responses.jsonl"
with open(cot_filename, 'w', encoding='utf-8') as outfile:
    for idx, training_example in enumerate(dataset['train']):
        print(idx)
        messages = generate_COT_prompt(training_example['sentence'], relation_names[training_example['relation']])
        completion = client.chat.completions.create(model= "llama3-8b-8192",
                                                        messages = messages,
                                                        temperature= 0)
        response = completion.choices[0].message.content
        json.dump({"sentence": training_example['sentence'], "response": response}, outfile)
        outfile.write('\n')

6660
6661
6662
6663
6664
6665
6666
6667
6668
6669
6670
6671
6672
6673
6674
6675
6676
6677
6678
6679
6680
6681
6682
6683
6684
6685
6686
6687
6688
6689
6690
6691
6692
6693
6694
6695
6696
6697
6698
6699
6700
6701
6702
6703
6704
6705
6706
6707
6708
6709
6710
6711
6712
6713
6714
6715
6716
6717
6718
6719
6720
6721
6722
6723
6724
6725
6726
6727
6728
6729
6730
6731
6732
6733
6734
6735
6736
6737
6738
6739
6740
6741
6742
6743
6744
6745
6746
6747
6748
6749
6750
6751
6752
6753
6754
6755
6756
6757
6758
6759
6760
6761
6762
6763
6764
6765
6766
6767
6768
6769
6770
6771
6772
6773
6774
6775
6776
6777
6778
6779
6780
6781
6782
6783
6784
6785
6786
6787
6788
6789
6790
6791
6792
6793
6794
6795
6796
6797
6798
6799
6800
6801
6802
6803
6804
6805
6806
6807
6808
6809
6810
6811
6812
6813
6814
6815
6816
6817
6818
6819
6820
6821
6822
6823
6824
6825
6826
6827
6828
6829
6830
6831
6832
6833
6834
6835
6836
6837
6838
6839
6840
6841
6842
6843
6844
6845
6846
6847
6848
6849
6850
6851
6852
6853
6854
6855
6856
6857
6858
6859


In [67]:
#Generate embeddings for training examples

import re
import pandas as pd
from sentence_transformers import SentenceTransformer

def get_entities_in_tags(sentence):
    pattern = r"<e1>(.*?)</e1>|<e2>(.*?)</e2>"

    matches = re.findall(pattern, sentence)

    words = [word for tpl in matches for word in tpl if word]
    return words[0], words[1]

model = SentenceTransformer("avsolatorio/GIST-small-Embedding-v0")

print("Embedding sentences from jsonl...")
cot_examples = []
embeddings = []
with open(cot_filename, 'r', encoding='utf-8') as file:
    for line in file:
        data = json.loads(line)
        content, sentence = data['response'], data['sentence']
        e1, e2 = get_entities_in_tags(sentence)
        content = f"The relation between '{e1}' and '{e2}' in the sentence '{sentence}':\n\n{content}"
        cot_examples.append(content)
        embeddings.append(model.encode(content))
        print(len(cot_examples))
print("Done.")

data = {'Content': cot_examples, 'Embedding': embeddings}
cot_df = pd.DataFrame(data)

Embedding sentences from jsonl...
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
26

1855
1856
1857
1858
1859
1860
1861
1862
1863
1864
1865
1866
1867
1868
1869
1870
1871
1872
1873
1874
1875
1876
1877
1878
1879
1880
1881
1882
1883
1884
1885
1886
1887
1888
1889
1890
1891
1892
1893
1894
1895
1896
1897
1898
1899
1900
1901
1902
1903
1904
1905
1906
1907
1908
1909
1910
1911
1912
1913
1914
1915
1916
1917
1918
1919
1920
1921
1922
1923
1924
1925
1926
1927
1928
1929
1930
1931
1932
1933
1934
1935
1936
1937
1938
1939
1940
1941
1942
1943
1944
1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046
2047
2048
2049
2050
2051
2052
2053
2054


3494
3495
3496
3497
3498
3499
3500
3501
3502
3503
3504
3505
3506
3507
3508
3509
3510
3511
3512
3513
3514
3515
3516
3517
3518
3519
3520
3521
3522
3523
3524
3525
3526
3527
3528
3529
3530
3531
3532
3533
3534
3535
3536
3537
3538
3539
3540
3541
3542
3543
3544
3545
3546
3547
3548
3549
3550
3551
3552
3553
3554
3555
3556
3557
3558
3559
3560
3561
3562
3563
3564
3565
3566
3567
3568
3569
3570
3571
3572
3573
3574
3575
3576
3577
3578
3579
3580
3581
3582
3583
3584
3585
3586
3587
3588
3589
3590
3591
3592
3593
3594
3595
3596
3597
3598
3599
3600
3601
3602
3603
3604
3605
3606
3607
3608
3609
3610
3611
3612
3613
3614
3615
3616
3617
3618
3619
3620
3621
3622
3623
3624
3625
3626
3627
3628
3629
3630
3631
3632
3633
3634
3635
3636
3637
3638
3639
3640
3641
3642
3643
3644
3645
3646
3647
3648
3649
3650
3651
3652
3653
3654
3655
3656
3657
3658
3659
3660
3661
3662
3663
3664
3665
3666
3667
3668
3669
3670
3671
3672
3673
3674
3675
3676
3677
3678
3679
3680
3681
3682
3683
3684
3685
3686
3687
3688
3689
3690
3691
3692
3693


5133
5134
5135
5136
5137
5138
5139
5140
5141
5142
5143
5144
5145
5146
5147
5148
5149
5150
5151
5152
5153
5154
5155
5156
5157
5158
5159
5160
5161
5162
5163
5164
5165
5166
5167
5168
5169
5170
5171
5172
5173
5174
5175
5176
5177
5178
5179
5180
5181
5182
5183
5184
5185
5186
5187
5188
5189
5190
5191
5192
5193
5194
5195
5196
5197
5198
5199
5200
5201
5202
5203
5204
5205
5206
5207
5208
5209
5210
5211
5212
5213
5214
5215
5216
5217
5218
5219
5220
5221
5222
5223
5224
5225
5226
5227
5228
5229
5230
5231
5232
5233
5234
5235
5236
5237
5238
5239
5240
5241
5242
5243
5244
5245
5246
5247
5248
5249
5250
5251
5252
5253
5254
5255
5256
5257
5258
5259
5260
5261
5262
5263
5264
5265
5266
5267
5268
5269
5270
5271
5272
5273
5274
5275
5276
5277
5278
5279
5280
5281
5282
5283
5284
5285
5286
5287
5288
5289
5290
5291
5292
5293
5294
5295
5296
5297
5298
5299
5300
5301
5302
5303
5304
5305
5306
5307
5308
5309
5310
5311
5312
5313
5314
5315
5316
5317
5318
5319
5320
5321
5322
5323
5324
5325
5326
5327
5328
5329
5330
5331
5332


6772
6773
6774
6775
6776
6777
6778
6779
6780
6781
6782
6783
6784
6785
6786
6787
6788
6789
6790
6791
6792
6793
6794
6795
6796
6797
6798
6799
6800
6801
6802
6803
6804
6805
6806
6807
6808
6809
6810
6811
6812
6813
6814
6815
6816
6817
6818
6819
6820
6821
6822
6823
6824
6825
6826
6827
6828
6829
6830
6831
6832
6833
6834
6835
6836
6837
6838
6839
6840
6841
6842
6843
6844
6845
6846
6847
6848
6849
6850
6851
6852
6853
6854
6855
6856
6857
6858
6859
6860
6861
6862
6863
6864
6865
6866
6867
6868
6869
6870
6871
6872
6873
6874
6875
6876
6877
6878
6879
6880
6881
6882
6883
6884
6885
6886
6887
6888
6889
6890
6891
6892
6893
6894
6895
6896
6897
6898
6899
6900
6901
6902
6903
6904
6905
6906
6907
6908
6909
6910
6911
6912
6913
6914
6915
6916
6917
6918
6919
6920
6921
6922
6923
6924
6925
6926
6927
6928
6929
6930
6931
6932
6933
6934
6935
6936
6937
6938
6939
6940
6941
6942
6943
6944
6945
6946
6947
6948
6949
6950
6951
6952
6953
6954
6955
6956
6957
6958
6959
6960
6961
6962
6963
6964
6965
6966
6967
6968
6969
6970
6971


In [68]:
# Save embeddings

from datasets import Dataset

cot_dataset = Dataset.from_pandas(cot_df)

cot_dataset.save_to_disk("cot_df.hf")

Saving the dataset (0/1 shards):   0%|          | 0/8000 [00:00<?, ? examples/s]

In [92]:
import random

def generatePromptFromTestExample(example):
    
    task_summary = "You are an expert annotator for a natural language proceessing task, performing relation extraction on sentences. You have the following relation labels: Cause-Effect(e1,e2), Cause-Effect(e2,e1), Component-Whole(e1,e2), Component-Whole(e2,e1), Content-Container(e1,e2), Content-Container(e2,e1), Entity-Destination(e1,e2), Entity-Destination(e2,e1), Entity-Origin(e1,e2), Entity-Origin(e2,e1), Instrument-Agency(e1,e2), Instrument-Agency(e2,e1), Member-Collection(e1,e2), Member-Collection(e2,e1), Message-Topic(e1,e2), Message-Topic(e2,e1), Product-Producer(e1,e2), Product-Producer(e2,e1), Other."
    example_sentence = example['sentence']
    examples = ""
    
    # K-shot examples (fetched)
    k_shot = 5
    random_shot = False
    
    if k_shot > 0:
        examples = ""
        if random_shot:
            samples = cot_dataset[random.sample(range(len(cot_dataset)), k)]
            for example_idx in range(k):
                examples += f"Example: {samples['Content'][example_idx]}\n\n"
        else:
            example_embedding = model.encode(example_sentence)
            scores, samples = cot_dataset.get_nearest_examples(
                "Embedding", example_embedding, k=k_shot
            )
            samples_df = pd.DataFrame.from_dict(samples)
            samples_df["Score"] = scores
            samples_df.sort_values("Score", ascending=True, inplace=True)

            intro_pattern = r"The relation between \S+ and \S+ in the sentence "
            for example in samples_df.iterrows():
                examples += f"Example: {re.sub(intro_pattern, '', example[1]['Content'][:3000])}\n\n"

    #tag_request = f"You are given the following sentence: {example['sentence']}. Respond with one sentence explaining your reasoning, followed by a label as your answer."
    tag_request = f"Example: '{example_sentence}''"

    #prompt = f"{task_summary}\n\n{tag_request}"
    prompt = f"{task_summary}\n\n{examples}\n\n{tag_request}"
    return prompt

cot_dataset.add_faiss_index(column="Embedding")

  0%|          | 0/8 [00:00<?, ?it/s]

Dataset({
    features: ['Content', 'Embedding'],
    num_rows: 8000
})

In [93]:
# Get LLM responses

eval_dataset = dataset['test'].select(range(0, 101))

outputs = []
for idx, example in enumerate(eval_dataset):
    prompt = generatePromptFromTestExample(example)
    chat_completion = client.chat.completions.create(model= "llama3-8b-8192",
                                                    messages= [{"role": "user", "content": prompt}],
                                                    temperature= 0)
    sentence = example["sentence"]
    response = chat_completion.choices[0].message.content
    outputs.append((sentence, response))
    print(idx)

output_file = "llama_eval_responses.jsonl"
with open("llama_eval_responses.jsonl", 'w', encoding='utf-8') as outfile:
    for output in outputs:
        json.dump({"sentence": output[0], "response": output[1]}, outfile)
        outfile.write('\n')

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100


In [95]:
# Evaluate responses

from sklearn.metrics import f1_score, accuracy_score

print("Getting sentences from jsonl...")

true = []
predicted = []

idx = 0
with open(output_file, 'r', encoding='utf-8') as file:
    for line in file:
        data = json.loads(line)
        sentence = data["sentence"]
        print(sentence)
        print("=" * 50)
        answer = data["response"]
        
        #find relation in answer
        stripped_answer = answer.replace(" ", "")
        predicted_label = None
        for relation_label in relation_names:
            if relation_label in stripped_answer:
                predicted_label = relation_label
        if predicted_label:
            predicted.append(relation_name_to_idx[predicted_label] - 1)
        else:
            predicted.append(0)
        
        print(answer)
        print("=" * 50)
        
        true.append(eval_dataset[idx]['relation'])
        true_relation_name = relation_names[true[idx]]#.split('(')[0]
        predicted_relation_name = relation_names[predicted[idx]]#.split('(')[0]
        true[idx] = true_relation_name
        predicted[idx] = predicted_relation_name
        print(f"Predicted: {predicted_relation_name}, True: {true_relation_name}")
        print("=" * 50)
        idx += 1
print("Done.")

print("Accuracy: ", accuracy_score(true, predicted))
print("Micro-F1: ", f1_score(true, predicted, average="micro"))
print("Macro-F1: ", f1_score(true, predicted, average="macro"))

Getting sentences from jsonl...
The most common <e1>audits</e1> were about <e2>waste</e2> and recycling.
Based on the provided examples, I will perform relation extraction on the given sentence.

The two entities in the sentence are "audits" and "waste". The sentence describes the topic of the audits, which is waste and recycling.

The interaction between the audits and the waste is that the audits are about the waste and recycling. The audits are the topic, and the waste and recycling are the subject matter being audited.

The correct relation extraction label for this sentence is Message-Topic(e1,e2). This label indicates that the entity "audits" (e1) is the message or topic, and the entity "waste" (e2) is the subject matter or topic being discussed.
Predicted: Message-Topic, True: Message-Topic
The <e1>company</e1> fabricates plastic <e2>chairs</e2>.
The two entities in the sentence are the "<e1>company</e1>" and the "<e2>chairs</e2>". The sentence describes the relationship between